In [1]:
import sys
sys.path.append("../Evaluation")

In [4]:
!pip install -e ../Maccabee > /dev/null

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [2]:
ls ../Evaluation/evaluation

__init__.py  __pycache__/  test.py


In [2]:
from evaluation.test import test

In [5]:
import numpy


True